In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc1


In [4]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cpu"

for train_index, test_index in kfold.split(np.arange(pos_num)):
    sampler = Sampler(res, train_index, test_index, null_mask)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.708908 auc:0.2903
epoch:  20 loss:0.258605 auc:0.9467
epoch:  40 loss:0.207434 auc:0.9515
epoch:  60 loss:0.195661 auc:0.9550
epoch:  80 loss:0.189174 auc:0.9579
epoch: 100 loss:0.183395 auc:0.9604
epoch: 120 loss:0.181069 auc:0.9613
epoch: 140 loss:0.179807 auc:0.9617
epoch: 160 loss:0.178726 auc:0.9621
epoch: 180 loss:0.178004 auc:0.9624
epoch: 200 loss:0.177672 auc:0.9627
epoch: 220 loss:0.175704 auc:0.9630
epoch: 240 loss:0.175105 auc:0.9633
epoch: 260 loss:0.173426 auc:0.9638
epoch: 280 loss:0.176518 auc:0.9643
epoch: 300 loss:0.168323 auc:0.9651
epoch: 320 loss:0.172148 auc:0.9646
epoch: 340 loss:0.163528 auc:0.9666
epoch: 360 loss:0.164702 auc:0.9663
epoch: 380 loss:0.160251 auc:0.9675
epoch: 400 loss:0.159448 auc:0.9675
epoch: 420 loss:0.155192 auc:0.9683
epoch: 440 loss:0.160638 auc:0.9660
epoch: 460 loss:0.165319 auc:0.9665
epoch: 480 loss:0.153059 auc:0.9689
epoch: 500 loss:0.150295 auc:0.9691
epoch: 520 loss:0.147820 auc:0.9697
epoch: 540 loss:0.146118 auc

In [5]:
true_datas.reset_index(drop=True).to_csv("true_gdsc1.csv")
predict_datas.reset_index(drop=True).to_csv("pred_gdsc1.csv")